## 讀取模型IGS

In [13]:
import OCC.Extend.DataExchange
from OCC.Core.IGESControl import IGESControl_Reader
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop_VolumeProperties
from OCC.Display.SimpleGui import init_display
from OCC.Extend.DataExchange import read_iges_file


#讀取模型
iges_file = r"C:\alan\ITRI\model\cube_10x10x10.IGS"
model = read_iges_file(iges_file)
# Get the top level shape
# model = iges_reader.OneShape()

# 初始化 3D 顯示環境
display, start_display, add_menu, add_function_to_menu = init_display()
display.DisplayShape(model, update=True)
start_display()


## 生成線段(input:兩點座標)

In [33]:
# 函數作用: 通過兩點生成線段
from OCC.Core.gp import gp_Pnt
from OCC.Core.GC import GC_MakeSegment
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire
from OCC.Display.SimpleGui import init_display

import numpy as np

# 輸入: 兩個點
# 輸出: 線段
# def generate_segment(point1, point2):
#     aSegment = GC_MakeSegment(gp_Pnt(*point1), gp_Pnt(*point2))
#     anEdge = BRepBuilderAPI_MakeEdge(aSegment.Value())
#     aWire = BRepBuilderAPI_MakeWire(anEdge.Edge())
#     return aWire.Shape()

point_start=np.array((-5,5,0))
point_end=np.array((5,5,0))

point_start=gp_Pnt(float(point_start[0]),float(point_start[1]), float(point_start[2]))
point_end=gp_Pnt(float(point_end[0]),float(point_end[1]), float(point_end[2]))
# print(point_start[0])
# p_start = gp_Pnt(point_start[0], point_start[1], point_start[2])
# p_end = gp_Pnt(point_start[0], point_start[1], point_start[2])

aSegment = GC_MakeSegment(point_start, point_end )
anEdge = BRepBuilderAPI_MakeEdge(aSegment.Value())
aWire = BRepBuilderAPI_MakeWire(anEdge.Edge())

#生成兩段線段
# E21=BRepBuilderAPI_MakeEdge(gp_Pnt(40,0,10),gp_Pnt(82.5,25,10)).Edge()
# E22=BRepBuilderAPI_MakeEdge(gp_Pnt(82.5,25,10),gp_Pnt(42.5,93,10)).Edge()
# W2=BRepBuilderAPI_MakeWire(E21,E22)

if __name__ == "__main__":
    display, start_display, add_menu, add_function_to_menu = init_display()
    display.DisplayShape(aWire.Shape(), update=True)
    # display.FitAll()
    start_display()

## 掃除範圍生成(input:起始點的法向量rxryrz)

In [9]:
#根據引導軌跡掃出圖形
from OCC.Core.gp import gp_Pnt, gp_Vec, gp_Dir
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire,BRepBuilderAPI_MakeFace
from OCC.Core.BRepOffsetAPI import BRepOffsetAPI_MakePipe
from OCC.Core.gp import gp_Circ, gp_Ax2, gp_Pnt, gp_Dir
from OCC.Display.SimpleGui import init_display
import numpy as np

#input
#normal_vector
normal_vector=np.array((0,1,0))
magnitude = np.linalg.norm(normal_vector)# 將向量除以其範數以得到單位向量
normal_vector = normal_vector/ magnitude

#two point_position
tan_vector=point_start-point_end
print(tan_vector)

#direction_1
dir_1= np.cross(normal_vector, tan_vector)
magnitude = np.linalg.norm(dir_1)# 將向量除以其範數以得到單位向量
dir_1 = dir_1/ magnitude

# 定義截面為方形
# 定義方形的四個頂點
point_1=point_start+normal_vector*0.5+dir_1*2
point_2=point_start+normal_vector*0.5-dir_1*2
point_3=point_start-normal_vector*0.5-dir_1*2
point_4=point_start-normal_vector*0.5+dir_1*2
p1 = gp_Pnt(point_1[0], point_1[1], point_1[2])
p2 = gp_Pnt(point_2[0], point_2[1], point_2[2])
p3 = gp_Pnt(point_3[0], point_3[1], point_3[2])
p4 = gp_Pnt(point_4[0], point_4[1], point_4[2])

# 使用這些頂點創建方形的邊緣
edge1 = BRepBuilderAPI_MakeEdge(p1, p2).Edge()
edge2 = BRepBuilderAPI_MakeEdge(p2, p3).Edge()
edge3 = BRepBuilderAPI_MakeEdge(p3, p4).Edge()
edge4 = BRepBuilderAPI_MakeEdge(p4, p1).Edge()

# 使用這些邊緣創建方形的線框
square_wire = BRepBuilderAPI_MakeWire(edge1, edge2, edge3, edge4).Wire()

# 將線框轉換為面，作為截面
F = BRepBuilderAPI_MakeFace(square_wire, True)
S=BRepOffsetAPI_MakePipe(aWire.Wire(), F.Shape())#第一个参数轨迹线,第二个参数是轮廟


display, start_display, add_menu, add_function_to_menu = init_display()  
display.DisplayShape(S.Shape(),update=True)
start_display()

[-10   0   0]


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Martin\anaconda3\envs\pyoccenv\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "c:\Users\Martin\anaconda3\envs\pyoccenv\lib\site-packages\OCC\Display\tkDisplay.py", line 62, in Zoom
    self._display.ZoomFactor(zoom_factor)
UnboundLocalError: local variable 'zoom_factor' referenced before assignment


## 對物件進行掃除

In [10]:
from OCC.Core.gp import gp_Pnt
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox, BRepPrimAPI_MakeSphere
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Cut
from OCC.Display.SimpleGui import init_display
from OCC.Core.IGESControl import IGESControl_Reader

# #讀取模型
# iges_file = r"C:\alan\ITRI\model\cube.IGS"
# iges_reader = read_iges_file(iges_file)
# # Get the top level shape
# model = iges_reader.OneShape()

# # 创建一个长方体
box = BRepPrimAPI_MakeBox(10, 10, 10).Shape()

# 创建一个几何切割对象
cut_algo = BRepAlgoAPI_Cut(box,S.Shape() )

# 执行几何切割操作
cut_result = cut_algo.Shape()

# 显示结果
display, start_display, add_menu, add_function_to_menu = init_display()

# display.DisplayShape(box, update=True)
# display.DisplayShape(sphere, update=True)
display.DisplayShape(cut_result, update=True)
start_display()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Martin\anaconda3\envs\pyoccenv\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "c:\Users\Martin\anaconda3\envs\pyoccenv\lib\site-packages\OCC\Display\tkDisplay.py", line 62, in Zoom
    self._display.ZoomFactor(zoom_factor)
UnboundLocalError: local variable 'zoom_factor' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Martin\anaconda3\envs\pyoccenv\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "c:\Users\Martin\anaconda3\envs\pyoccenv\lib\site-packages\OCC\Display\tkDisplay.py", line 62, in Zoom
    self._display.ZoomFactor(zoom_factor)
UnboundLocalError: local variable 'zoom_factor' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Martin\anaconda3\envs\pyoccenv\lib\tkinter\__init__.py", line 1892, in __call__


## 計算掃除體積

In [11]:
from OCC.Core.BRepGProp import brepgprop_VolumeProperties
from OCC.Core.GProp import GProp_GProps
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopoDS import TopoDS_Shape

def calculate_volume(shape):
    # 將 shape 轉換為 TopoDS_Shape
    topods_shape = TopoDS_Shape(shape)
    # 初始化體積屬性
    volume_props = GProp_GProps()

    # 計算體積屬性
    brepgprop_VolumeProperties(topods_shape, volume_props)

    # 獲取體積
    volume = volume_props.Mass()
    
    return volume

# 假設您已經有一個名為 model 的形狀
volume_origin = calculate_volume(model)
volume_cut_result=calculate_volume(cut_result)
print("Model Volume:", volume_origin) 
print("Model Volume:", volume_cut_result)

Model Volume: -666.6666666666669
Model Volume: 989.9999999999999


C:\Users\Martin\AppData\Local\Temp\ipykernel_2196\1102814728.py:13: DeprecationWarning: Call to deprecated function brepgprop_VolumeProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.VolumeProperties
  brepgprop_VolumeProperties(topods_shape, volume_props)
C:\Users\Martin\AppData\Local\Temp\ipykernel_2196\1102814728.py:13: DeprecationWarning: Call to deprecated function brepgprop_VolumeProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.VolumeProperties
  brepgprop_VolumeProperties(topods_shape, volume_props)


In [9]:
import OCC.Extend.DataExchange
from OCC.Core.IGESControl import IGESControl_Reader
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop_VolumeProperties
from OCC.Display.SimpleGui import init_display


model_volume = calculate_volume(model)
print("Model Volume:", model_volume)

Model Volume: -666.6666666666669


C:\Users\Martin\AppData\Local\Temp\ipykernel_24632\1102814728.py:13: DeprecationWarning: Call to deprecated function brepgprop_VolumeProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.VolumeProperties
  brepgprop_VolumeProperties(topods_shape, volume_props)


## 計算掃除面積

In [10]:
from OCC.Core.BRepGProp import brepgprop_SurfaceProperties
from OCC.Core.GProp import GProp_GProps
from OCC.Core.TopoDS import TopoDS_Shape

def calculate_surface_area(shape):
    # 將 shape 轉換為 TopoDS_Shape
    topods_shape = TopoDS_Shape(shape)

    # 初始化表面屬性
    surface_props = GProp_GProps()

    # 計算表面屬性
    brepgprop_SurfaceProperties(topods_shape, surface_props)
    

    # 獲取表面積
    surface_area = surface_props.Mass()

    return surface_area


surface_origin = calculate_surface_area(model)
surface_cut_result=calculate_surface_area(cut_result)
print("Model Volume:", surface_origin) 
print("Model Volume:", surface_cut_result)

Model Volume: 600.0
Model Volume: 619.9999999999999


C:\Users\Martin\AppData\Local\Temp\ipykernel_24632\3613906193.py:13: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(topods_shape, surface_props)
C:\Users\Martin\AppData\Local\Temp\ipykernel_24632\3613906193.py:13: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(topods_shape, surface_props)
